In [48]:
# Initial imports
import os
import json
import requests
import pandas as pd
import numpy as np
import panel as pn
from panel.interact import interact
import plotly.express as px
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline
pn.extension("plotly")

In [49]:
# Load .env enviroment variables
load_dotenv()

True

In [50]:
# Set Alpaca API key and secret
alpaca_api = os.getenv("ALPACA_API")
alpaca_secret = os.getenv("ALPACA_SECRET")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api,
    alpaca_secret,
    api_version = "v2"
)

In [36]:
# Format current date as ISO format
start_date = pd.Timestamp("2018-12-31", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-03-06", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["FB", "AMZN", "AAPL", "NFLX", "GOOG", "SHOP", "PLD", "TTWO", "ATVI", "NVDA", "AMD",
          "PEP", "MKC", "DPZ", "WING", "COST", "ETSY", "CPB", "OKTA", "ZM"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get price data for 20 picks 
ticker_data = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

# Drop any rows with NaN
ticker_data.dropna(inplace=True)
ticker_data.isna().sum()


# Preview DataFrame
ticker_data.head(20)



AAPL                                       \
                              open      high      low   close    volume   
time                                                                      
2019-04-18 00:00:00-04:00  203.120  204.1500  202.520  203.86  21928367   
2019-04-22 00:00:00-04:00  202.830  204.9400  202.340  204.64  13720923   
2019-04-23 00:00:00-04:00  204.430  207.7500  203.900  207.51  19401417   
2019-04-24 00:00:00-04:00  207.360  208.4800  207.050  207.18  14914939   
2019-04-25 00:00:00-04:00  206.830  207.7600  205.120  205.24  15908807   
2019-04-26 00:00:00-04:00  204.900  205.0000  202.120  204.29  16315669   
2019-04-29 00:00:00-04:00  204.400  205.9700  203.860  204.61  19641066   
2019-04-30 00:00:00-04:00  203.060  203.4000  199.110  200.57  35362106   
2019-05-01 00:00:00-04:00  209.880  215.3100  209.230  210.52  57751414   
2019-05-02 00:00:00-04:00  209.840  212.6500  208.130  209.17  29014844   
2019-05-03 00:00:00-04:00  210.740  211.8400  210.230  211.78  17987793   
2019-05-06 00:00:00-04:00  204.290  208.8400  203.500  208.60  28949691   
2019-05-07 00:00:00-04:00  205.880  207.4175  200.825  202.86  34328425   
2019-05-08 00:00:00-04:00  201.900  205.3400  201.750  202.90  22729670   
2019-05-09 00:00:00-04:00  200.400  201.6800  196.660  200.72  32427147   
2019-05-10 00:00:00-04:00  197.419  198.8500  192.770  197.30  36118438   
2019-05-13 00:00:00-04:00  187.710  189.4800  182.850  185.72  50491792   
2019-05-14 00:00:00-04:00  186.410  189.7000  185.410  188.65  31498214   
2019-05-15 00:00:00-04:00  186.270  191.7500  186.020  190.92  23655393   
2019-05-16 00:00:00-04:00  189.910  192.4689  188.840  190.14  28660083   

                             AMD                                        ...  \
                            open     high      low    close     volume  ...   
time                                                                    ...   
2019-04-18 00:00:00-04:00  27.60  27.8800  27.3400  27.6767   37302337  ...   
2019-04-22 00:00:00-04:00  27.62  28.2300  27.3900  28.1700   33846781  ...   
2019-04-23 00:00:00-04:00  28.18  28.4900  27.7900  27.9700   37974538  ...   
2019-04-24 00:00:00-04:00  28.10  28.8500  27.9300  28.4600   48395877  ...   
2019-04-25 00:00:00-04:00  28.65  28.8600  27.3600  27.6500   51487295  ...   
2019-04-26 00:00:00-04:00  27.62  27.9000  27.0500  27.8800   44699014  ...   
2019-04-29 00:00:00-04:00  27.90  28.1400  27.5000  27.6811   40385475  ...   
2019-04-30 00:00:00-04:00  27.59  27.8000  26.9400  27.6300   54019126  ...   
2019-05-01 00:00:00-04:00  28.95  29.1500  26.7800  26.8000  127287384  ...   
2019-05-02 00:00:00-04:00  26.94  28.6400  26.6100  28.2900   95312594  ...   
2019-05-03 00:00:00-04:00  28.30  28.4201  27.6600  28.2200   51718501  ...   
2019-05-06 00:00:00-04:00  26.72  27.5000  26.4500  27.4300   64760337  ...   
2019-05-07 00:00:00-04:00  27.20  27.3500  26.2100  26.6600   70007742  ...   
2019-05-08 00:00:00-04:00  26.41  27.7100  26.2700  27.0900   62428679  ...   
2019-05-09 00:00:00-04:00  26.70  27.3800  26.0300  27.2000   69825507  ...   
2019-05-10 00:00:00-04:00  27.03  28.1000  26.9300  27.9500   79121383  ...   
2019-05-13 00:00:00-04:00  26.98  27.2300  26.1000  26.2400   92488651  ...   
2019-05-14 00:00:00-04:00  26.53  27.4800  26.1500  27.3100   76955796  ...   
2019-05-15 00:00:00-04:00  26.87  27.7900  26.7300  27.5800   52483356  ...   
2019-05-16 00:00:00-04:00  27.37  28.3700  27.2699  28.0200   60976490  ...   

                            WING                                       ZM  \
                            open     high    low  close     volume   open   
time                                                                        
2019-04-18 00:00:00-04:00  76.08  78.4500  75.90  77.95   318224.0  65.00   
2019-04-22 00:00:00-04:00  77.53  78.8700  76.30  77.04   318655.0  61.00   
2019-04-23 00:00:00-04:00  77.05  78.4700  76.53  77.22   362748.0  66.87   
2019-04-24 00:00:00-04:0

In [53]:
# Create baseline dataframe from 2019 dataset
baseline_df = ticker_data.iloc[:253]
baseline_df

# Drop the time from date index
baseline_df.index = baseline_df.index.date

# Create closing prices Data frame from 2019 baseline data
closing_df = pd.DataFrame()

closing_df["AAPL"]["close"] = baseline_df["AAPL"]["close"]
closing_df["AMD"]["close"] = baseline_df["AMD"]["close"]
closing_df["AMZN"]["close"] = baseline_df["AMZN"]["close"]
closing_df["ATVI"]["close"] = baseline_df["ATVI"]["close"]
closing_df["COST"]["close"] = baseline_df["COST"]["close"]
closing_df["CPB"]["close"] = baseline_df["CPB"]["close"]
closing_df["DPZ"]["close"] = baseline_df["DPZ"]["close"]
closing_df["ETSY"]["close"] = baseline_df["ETSY"]["close"]
closing_df["FB"]["close"] = baseline_df["FB"]["close"]
closing_df["GOOG"]["close"] = baseline_df["GOOG"]["close"]
closing_df["MKC"]["close"] = baseline_df["MKC"]["close"]
closing_df["NFLX"]["close"] = baseline_df["NFLX"]["close"]
closing_df["NVDA"]["close"] = baseline_df["NVDA"]["close"]
closing_df["OKTA"]["close"] = baseline_df["OKTA"]["close"]
closing_df["SHOP"]["close"] = baseline_df["SHOP"]["close"]
closing_df["PEP"]["close"] = baseline_df["PEP"]["close"]
closing_df["PLD"]["close"] = baseline_df["PLD"]["close"]
closing_df["TTWO"]["close"] = baseline_df["TTWO"]["close"]
closing_df["WING"]["close"] = baseline_df["WING"]["close"]
closing_df["ZM"]["close"] = baseline_df["ZM"]["close"]

# Add Volume column to 

# Display sample data
closing_df.head(10)



SyntaxError: invalid syntax (<ipython-input-53-3b2de8fbcb7b>, line 14)

In [ ]:
# Create closing data charts from 2019


In [38]:
# Create daily high prices Data frame from 2019 baseline data
high_df = pd.DataFrame()

high_df["AAPL"] = baseline_df["AAPL"]["high"]
high_df["AMD"] = baseline_df["AMD"]["high"]
high_df["AMZN"] = baseline_df["AMZN"]["high"]
high_df["ATVI"] = baseline_df["ATVI"]["high"]
high_df["COST"] = baseline_df["COST"]["high"]
high_df["CPB"] = baseline_df["CPB"]["high"]
high_df["DPZ"] = baseline_df["DPZ"]["high"]
high_df["ETSY"] = baseline_df["ETSY"]["high"]
high_df["FB"] = baseline_df["FB"]["high"]
high_df["GOOG"] = baseline_df["GOOG"]["high"]
high_df["MKC"] = baseline_df["MKC"]["high"]
high_df["NFLX"] = baseline_df["NFLX"]["high"]
high_df["NVDA"] = baseline_df["NVDA"]["high"]
high_df["OKTA"] = baseline_df["OKTA"]["high"]
high_df["SHOP"] = baseline_df["SHOP"]["high"]
high_df["PEP"] = baseline_df["PEP"]["high"]
high_df["PLD"] = baseline_df["PLD"]["high"]
high_df["TTWO"] = baseline_df["TTWO"]["high"]
high_df["WING"] = baseline_df["WING"]["high"]
high_df["ZM"] = baseline_df["ZM"]["high"]

high_df.head(10)

,AAPL,AMD,AMZN,ATVI,COST,CPB,DPZ,ETSY,FB,GOOG,MKC,NFLX,NVDA,OKTA,SHOP,PEP,PLD,TTWO,WING,ZM
2019-04-18,204.15,27.88,1870.820,45.59,247.250,39.605,270.8500,64.2200,178.8800,1242.0000,154.85,360.4100,188.9300,92.03,220.7800,128.2600,73.685,91.27,78.4500,66.000
2019-04-22,204.94,28.23,1888.420,45.53,245.770,39.580,268.6700,63.7800,181.6650,1249.0900,154.59,377.6900,189.1000,95.58,225.9900,127.2400,73.540,91.99,78.8700,68.900
2019-04-23,207.75,28.49,1929.260,46.76,246.530,39.500,271.6350,65.5600,184.2200,1269.0000,153.06,384.8000,191.9300,101.06,227.3900,127.5400,74.430,94.24,78.4700,74.169
2019-04-24,208.48,28.85,1929.690,46.89,248.700,39.485,302.0450,66.5300,185.1400,1268.0100,152.34,381.9000,192.8100,103.29,224.6000,127.9800,75.350,93.85,78.7000,71.500
2019-04-25,207.76,28.86,1922.450,47.46,247.150,38.870,282.4625,68.2112,198.4800,1267.4083,151.50,374.7600,190.4500,104.98,222.1900,126.3999,75.660,94.22,78.0975,66.850
2019-04-26,205.00,27.90,1951.000,48.48,245.890,38.660,279.3900,68.4300,192.9000,1273.0700,152.87,375.1400,180.8876,103.70,225.1800,127.2450,76.130,96.34,78.4600,66.990
2019-04-29,205.97,28.14,1956.335,48.98,245.750,38.730,273.7900,69.6400,195.4100,1289.2700,153.78,374.5753,180.4500,104.80,228.7402,126.9600,76.349,98.37,77.4500,68.500
2019-04-30,203.40,27.80,1935.710,48.65,245.790,38.720,273.2350,69.7850,197.3900,1192.8100,154.69,374.5000,182.0300,104.70,247.5000,128.3900,76.820,97.99,77.4400,72.520
2019-05-01,215.31,29.15,1943.640,49.56,245.460,38.780,274.6600,68.7200,196.1769,1188.0500,154.24,385.9900,184.8100,105.50,252.3015,128.0000,78.560,100.34,76.1475,76.950
2019-05-02,212.65,28.64,1921.550,49.83,242.885,38.020,274.4700,68.0950,194.0000,1174.1895,153.54,383.5000,184.8700,104.59,260.8200,127.5500,77.990,100.88,75.8500,75.890


In [39]:
# Create daily high prices Data frame from 2019 baseline data
volume_df = pd.DataFrame()

volume_df["AAPL"] = baseline_df["AAPL"]["volume"]
volume_df["AMD"] = baseline_df["AMD"]["volume"]
volume_df["AMZN"] = baseline_df["AMZN"]["volume"]
volume_df["ATVI"] = baseline_df["ATVI"]["volume"]
volume_df["COST"] = baseline_df["COST"]["volume"]
volume_df["CPB"] = baseline_df["CPB"]["volume"]
volume_df["DPZ"] = baseline_df["DPZ"]["volume"]
volume_df["ETSY"] = baseline_df["ETSY"]["volume"]
volume_df["FB"] = baseline_df["FB"]["volume"]
volume_df["GOOG"] = baseline_df["GOOG"]["volume"]
volume_df["MKC"] = baseline_df["MKC"]["volume"]
volume_df["NFLX"] = baseline_df["NFLX"]["volume"]
volume_df["NVDA"] = baseline_df["NVDA"]["volume"]
volume_df["OKTA"] = baseline_df["OKTA"]["volume"]
volume_df["SHOP"] = baseline_df["SHOP"]["volume"]
volume_df["PEP"] = baseline_df["PEP"]["volume"]
volume_df["PLD"] = baseline_df["PLD"]["volume"]
volume_df["TTWO"] = baseline_df["TTWO"]["volume"]
volume_df["WING"] = baseline_df["WING"]["volume"]
volume_df["ZM"] = baseline_df["ZM"]["volume"]

volume_df.head(10)

,AAPL,AMD,AMZN,ATVI,COST,CPB,DPZ,ETSY,FB,GOOG,MKC,NFLX,NVDA,OKTA,SHOP,PEP,PLD,TTWO,WING,ZM
2019-04-18,21928367,37302337,2613709,3916981,1130721,1305430.0,893644.0,1696019,9740923,1118597,597880.0,8049911,8621013,1843911,1250005,6034253,1991482,1683121,318224.0,29003894.0
2019-04-22,13720923,33846781,3196934,3266896,1083705,1235130.0,1689177.0,923690,11672646,656033,763879.0,11282629,6046367,1638040,1294357,2823811,2274652,1461788,318655.0,9559767.0
2019-04-23,19401417,37974538,4123487,5383188,1322147,1391111.0,1650056.0,1654254,16649460,1112768,664561.0,9304333,7409773,3331240,1799240,3201374,1779789,1375871,362748.0,6559656.0
2019-04-24,14914939,48395877,3044542,4689481,939562,1388297.0,3583736.0,1071222,23160514,695256,570090.0,6111210,7074012,2402154,1997850,2504971,1716547,773937,411489.0,4765074.0
2019-04-25,15908807,51487295,4154636,6783848,1048078,2082707.0,1317710.0,1492943,47630063,955607,503153.0,5871934,10727204,1901188,1645146,3671826,1574713,808761,327372.0,2949258.0
2019-04-26,16315669,44699014,7614548,5823222,1455811,1325179.0,581199.0,962471,19839970,1079506,443226.0,4810395,20542879,1167029,1361025,2564476,2018085,1312805,287277.0,1490694.0
2019-04-29,19641066,40385475,3542692,6699827,1190172,1234486.0,786899.0,1238257,18176943,1702119,484216.0,3560763,8919651,1282643,1884039,2205241,1926937,2306309,367855.0,1784513.0
2019-04-30,35362106,54019126,3106108,6025167,1069725,1278448.0,660239.0,2024410,20556973,5730819,688015.0,3467914,8310874,1295623,5997014,3629294,1732369,1092979,311894.0,3611440.0
2019-05-01,57751414,127287384,2814558,8874571,1125283,1425351.0,383461.0,1072615,14421521,2458890,526016.0,8709382,8264646,1619852,3115239,2873720,3227733,2742394,292748.0,3226692.0
2019-05-02,29014844,95312594,3641665,12143533,1103548,1165681.0,517369.0,1271097,11924746,1795505,559427.0,5034699,9268721,1590112,4016300,2497094,2439092,1210707,468322.0,2441372.0


In [52]:
aapl_vol = px.line(volume_df["AAPL"], title="Apple Volume")
aapl_avg = px.line(volume_df["AAPL"].rolling(10).mean(), title="Apple Avg. Volume")
amd_vol = px.line(volume_df["AMD"], title="AMD Volume")

tabs = pn.Tabs(('Apple Volume',aapl_vol), ("AMD Volume", amd_vol))
tabs

Tabs
    [0] Plotly(Figure, name='Apple Volume')
    [1] Plotly(Figure, name='AMD Volume')